In [ ]:
import os 
import glob
import pandas as pd
from datetime import datetime
import datetime as dt

In [ ]:
sales_checked_path = "D:\\Projects\\Colgate\\Hills\\2019\\Stackline\\Amazon\\Sales\\checked"
promo_path = "D:\\Projects\\Colgate\\Hills\\2019\\Stackline\\Amazon\\Promo"
history_promo_data_folder= "D:\\Projects\\Colgate\\Hills\\2019\\Stackline\\Amazon\\"
sns_data_path = "D:\\Projects\\Colgate\\Hills\\2019\\OneHub_files"
map_folder = "D:\\Projects\\Colgate\\Hills\\2019\\Mapping\\"

In [ ]:
to_save = "D:\\Projects\\Colgate\\Hills\\2019\\19_07_09\\output_of_code\\"

In [ ]:
def multiindex_pivot(df, columns=None, values=None):
    names = list(df.index.names)
    df = df.reset_index()
    list_index = df[names].values
    tuples_index = [tuple(i) for i in list_index] # hashable
    df = df.assign(tuples_index=tuples_index)
    df = df.pivot(index="tuples_index", columns=columns, values=values)
    tuples_index = df.index  # reduced
    index = pd.MultiIndex.from_tuples(tuples_index, names=names)
    df.index = index
    return df

In [ ]:
#combine all files in the list
#combined_csv = pd.concat([pd.read_csv(f, encoding='latin-1', sep=",") for f in all_filenames ])

# Read All Sales and Promo Files

In [ ]:
os.chdir(sales_checked_path)
extension = 'csv'
all_filenames_raw = [i for i in glob.glob('*.{}'.format(extension))]
all_filenames = [k for k in all_filenames_raw if 'amazon' in k]
print(len(all_filenames))
merged_files = pd.DataFrame()
for f in all_filenames:
    df_0 = pd.read_csv(f, encoding='latin-1')
    df_0["source"] = f
    merged_files = pd.concat([merged_files,df_0])
amazon_sales_file = merged_files

In [ ]:
all_filenames_1 = [k for k in all_filenames_raw if 'Sales_' in k]
print(len(all_filenames_1))
merged_files = pd.DataFrame()
for f in all_filenames_1:
    df_0 = pd.read_csv(f, encoding='latin-1', sep=";")
    df_0["source"] = f
    merged_files = pd.concat([merged_files,df_0])
sales_file = merged_files

In [ ]:
sales_file_5 = sales_file[sales_file["source"]=="Stackline_Atlas_Sales_05.csv"]
sales_file_not_5 = sales_file[sales_file["source"]!="Stackline_Atlas_Sales_05.csv"]

In [ ]:
# not 201714, 201715, 217016
sales_file_5_sev_weeks = sales_file_5[~((sales_file_5["weekid"]== 201714) | (sales_file_5["weekid"]== 201715) | (sales_file_5["weekid"]== 201716))]
#set(sales_file_5_sev_weeks["weekid"])

In [ ]:
sales_file_5_sev_weeks = sales_file_5[~sales_file_5["weekid"].isin([201714, 201715, 201716])]
#set(sales_file_5_sev_weeks["weekid"])

In [ ]:
sales_file = pd.concat([sales_file_not_5,sales_file_5_sev_weeks])

In [ ]:
os.chdir(promo_path)
all_filenames_0 = [i for i in glob.glob('*.{}'.format(extension))]
#all_filenames_1 = [k for k in all_filenames_0 if 'Sales' in k]
print(len(all_filenames_0))
merged_files = pd.DataFrame()
for f in all_filenames_0:
    df_0 = pd.read_csv(f, encoding='latin-1')
    df_0["source"] = f
    merged_files = pd.concat([merged_files,df_0])
month_promo_file = merged_files

In [ ]:
history_promo_data = pd.read_csv(history_promo_data_path + "hills_atlas_promo_history_UntilMay2019.csv")
history_promo_data["source"] = "hills_atlas_promo_history_UntilMay2019.csv"

# Filter columns

In [ ]:
sales_file_1=sales_file[['retailersku', 'weekid', 'volume', 'estimatesales',"title"]]
amazon_sales_file_1=amazon_sales_file[[ 'retailersku', 'weekid', 'volume', 'estimatesales',"title"]]
all_data_sales = pd.concat([sales_file_1, amazon_sales_file_1])
all_data_sales.drop_duplicates(inplace=True)
all_data_sales["weekid"] = all_data_sales["weekid"].astype("str")

# Create map for all skus with the longest title

In [ ]:
title_sku_df =all_data_sales[["retailersku","title"]]
title_sku_df["len_title"] = title_sku_df.title.apply(lambda x: len(x))
title_sku_df.head()
max_len_of_title_sku_df = title_sku_df[["len_title", "retailersku"]]
max_len_of_title_sku_df = max_len_of_title_sku_df.groupby("retailersku").max()
max_len_of_title_sku_df.head()
max_len_of_title_sku_df.reset_index(inplace = True)
uniq_title_sku = title_sku_df.merge(max_len_of_title_sku_df, on = ["retailersku", "len_title"], how="inner")
uniq_title_sku["is_duplicated_sku"] = uniq_title_sku["retailersku"].duplicated()
uniq_title_sku
uniq_title_sku = uniq_title_sku[uniq_title_sku["is_duplicated_sku"] == False]
map_title = uniq_title_sku[["retailersku", "title"]]
map_title.rename(columns={'title': 'title_from_sales'}, inplace=True)
map_title.head()
map_title.to_csv(to_save+"map_title_all_sku.csv")
all_data_sales = all_data_sales[['retailersku', 'weekid', 'volume', 'estimatesales']]

# Mapping for attributes

In [ ]:
map_atributes = pd.read_csv(map_folder+"Stackline_attribute_v3.csv", encoding="latin-1")
map_atributes.drop_duplicates(inplace=True)
map_atributes = map_atributes[map_atributes["retailer"]=="Amazon"]
map_atributes = map_atributes[['manufacturer', 'brand', 'Title',  'retailersku', 'species_form', 'bag_size', 'lifestage', 'usage']]
print(map_atributes.isna().sum())
map_atributes.fillna('Unknown',inplace=True)

# Map manufacture and brand to general

In [ ]:
map_manuf_brand_to_general = pd.read_csv(map_folder+"map_manuf_brand_to_general.csv")
map_atributes = map_atributes.merge(map_manuf_brand_to_general, on = ['manufacturer', 'brand'], how = "left")
print(map_atributes.isnull().sum())
map_atributes["manufacturer_general"].fillna("A/O MFG", inplace = True)
map_atributes["brand_general"].fillna("A/O BRAND", inplace = True)

# Mapping foodclass

In [ ]:
map_foodclass = pd.read_csv(map_folder+"map_foodclass.csv")

In [ ]:
for index, row in map_foodclass.iterrows():
    previous_value = list(row)[2]
    new_value = list(row)[3]  
    map_atributes.species_form[map_atributes.species_form == previous_value] = new_value
print("   map_atributes:")    
for col in list(map_atributes.columns):
    print("len of "+col+": "+str(len(set(map_atributes[col]))))
print("   map_manuf_brand_to_general:")    
for col in list(map_manuf_brand_to_general.columns):
    print("len of "+col+": "+str(len(set(map_atributes[col]))))

map_title_attr = map_atributes
map_atributes.to_csv(to_save+"map_atributes.csv")

# Mapping size

In [ ]:
map_size = pd.read_csv(map_folder+"map_size.csv")

# Drop treats

In [ ]:
all_data_sales["promo_type"]="None"
all_data_sales=all_data_sales.merge(map_title, on="retailersku")
all_data_sales["retailersku"].replace(to_replace="0789331144", value="789331144", inplace = True)
all_data_sales_with_attr = all_data_sales.merge(map_atributes, on="retailersku", how="left")
all_data_sales_without_treats = all_data_sales_with_attr[~(all_data_sales_with_attr["species_form"].str.contains("drop", na=False))]
all_data_sales_with_treats = all_data_sales_with_attr[(all_data_sales_with_attr["species_form"].str.contains("drop", na=False))]
len(all_data_sales_without_treats)

In [ ]:
all_data_sales_without_treats.estimatesales.sum()

## Check&CSV

In [ ]:
print(all_data_sales.volume.sum()-all_data_sales_with_attr.volume.sum())
print(all_data_sales_without_treats.volume.sum()+all_data_sales_with_treats.volume.sum()-all_data_sales_with_attr.volume.sum())

In [ ]:
all_data_sales_without_treats.to_csv(to_save+"all_data_sales_without_treats.csv")

# Final Sales

In [ ]:
final_data_sales = all_data_sales_without_treats[["Title",'title_from_sales',"retailersku",'manufacturer', 'brand','manufacturer_general', 'brand_general', 'species_form',
                                            'bag_size', 'lifestage', 'usage',"promo_type", 'weekid',  'volume', 'estimatesales']]
final_data_sales.rename(columns={'volume':"Sale Units", 'estimatesales':"Total Sales" },inplace = True)
#drop unknown
#final_data_sales.dropna(inplace=True)
final_data_sales.fillna("Unknown", inplace=True)
final_data_sales_grouped = final_data_sales.groupby(['Title','title_from_sales',"retailersku",'manufacturer', 'brand','manufacturer_general', 'brand_general', 
                                                     'species_form', 'bag_size', 'lifestage', 'usage',"promo_type", "weekid"]).max()
final_data_sales_grouped["AvgPrice per Unit"]=final_data_sales_grouped["Total Sales"]/final_data_sales_grouped["Sale Units"]
final_data_sales_grouped.reset_index(inplace =True)
final_data_sales_grouped.head(1)

In [ ]:
final_data_sales_grouped["Total Sales"].sum()

In [ ]:
final_data_sales_grouped_EQ = final_data_sales_grouped.merge(map_size, on="retailersku", how="left")
final_data_sales_grouped_EQ[['Size', 'lb/oz', 'Size lb', 'count']] = final_data_sales_grouped_EQ[['Size', 'lb/oz', 'Size lb', 'count']].fillna("Unknown")
final_data_sales_grouped_EQ = final_data_sales_grouped_EQ[['Title', 'title_from_sales', 'retailersku', 'manufacturer', 'brand',
       'manufacturer_general', 'brand_general', 'species_form', 'bag_size',
       'lifestage', 'usage',
       'Size', 'lb/oz', 'Size lb', 'count', 'coef', 'promo_type', 'weekid', 
       'Total Sales','Sale Units', 'AvgPrice per Unit' ]]
final_data_sales_grouped_EQ["Sale Volume"] = final_data_sales_grouped_EQ["Sale Units"]*final_data_sales_grouped_EQ["coef"]
final_data_sales_grouped_EQ["AvgPrice per lb"]=final_data_sales_grouped_EQ["Total Sales"]/final_data_sales_grouped_EQ["Sale Volume"]
final_data_sales_grouped_EQ.fillna(0,inplace=True)

# Check&CSV

In [ ]:
print(final_data_sales["Sale Units"].sum()-all_data_sales_without_treats.volume.sum())
#next not 0
print(final_data_sales_grouped["Sale Units"].sum()-all_data_sales_without_treats.volume.sum()+15016) 
print(final_data_sales_grouped["Total Sales"].sum()-all_data_sales_without_treats.estimatesales.sum()+596016.5400018692)
#print(final_data_grouped['SNS Units'].sum()-all_data_without_treats['SNS Units'].sum())
#print(final_data_grouped['Promo Sales'].sum()-promo_to_all_file['Promo Sales'].sum())

In [ ]:
final_data_sales_grouped_EQ.to_csv(to_save+"final_data_grouped_EQ.csv")

# Promo Final

In [ ]:
map_promo = pd.read_csv(map_folder+"map_promo.csv", encoding="latin-1")
month_promo_file_1=month_promo_file[['retailersku', 'weekid', 'promotype', 'retailsales', 'unitssold']]
history_promo_data_1=history_promo_data[['retailersku', 'weekid', 'promotype', 'retailsales','unitssold']]
all_data_promo = pd.concat([month_promo_file_1, history_promo_data_1])
all_data_promo = all_data_promo.merge(map_title, how="left", on="retailersku")
all_data_promo["weekid"] = all_data_promo["weekid"].astype("str")
all_data_promo = all_data_promo.merge(map_promo, how="left", on="promotype")
all_data_promo.drop("promotype",axis=1, inplace=True)
all_data_promo.drop_duplicates(inplace=True)
all_data_promo.drop_duplicates(subset=["retailersku","weekid","promo_type"], keep="last",inplace=True)
# to drop treats
all_data_promo_uniq_title_attr = all_data_promo.merge(map_atributes, on = ["retailersku"],how ="left")
all_data_promo_without_treats = all_data_promo_uniq_title_attr[~(all_data_promo_uniq_title_attr["species_form"]=="drop")]
all_data_promo_with_treats = all_data_promo_uniq_title_attr[all_data_promo_uniq_title_attr["species_form"]=="drop"]

final_data_promo = all_data_promo_without_treats[['retailersku', 'Title',"title_from_sales",  
       'manufacturer', 'brand', 'species_form', 'bag_size', 'lifestage',
       'usage', 'manufacturer_general', 'brand_general' , 'promo_type',
       'weekid', 'retailsales',
        'unitssold']]
#final_data_promo.fillna("Unknown",inplace=True)
#Check
print(all_data_promo_without_treats.unitssold.sum()+all_data_promo_with_treats.unitssold.sum() - all_data_promo.unitssold.sum())
print(final_data_promo['unitssold'].sum() - all_data_promo_without_treats.unitssold.sum())
#Changes to merge with sales
final_data_promo.rename(columns={'unitssold':"Promo Units", 'retailsales':"Promo Sales" },inplace = True)
final_data_promo["retailersku"].replace(to_replace="0789331144", value="789331144", inplace = True)
#final_data_promo = final_data_promo[["retailersku","weekid","promo_type", "Promo Units", "Promo Sales"]]

In [ ]:
final_data_promo = final_data_promo[["Title",'title_from_sales',"retailersku",'manufacturer', 'brand','manufacturer_general', 'brand_general', 'species_form',
                                            'bag_size', 'lifestage', 'usage',"promo_type", 'weekid',  "Promo Units", "Promo Sales"]]
final_data_promo_grouped = final_data_promo.groupby(['Title','title_from_sales',"retailersku",'manufacturer', 'brand','manufacturer_general', 'brand_general', 
                                                     'species_form', 'bag_size', 'lifestage', 'usage',"promo_type", "weekid"]).max()
final_data_promo_grouped["Promo AvgPrice per Unit"]=final_data_promo_grouped["Promo Sales"]/final_data_promo_grouped["Promo Units"]
final_data_promo_grouped.reset_index(inplace =True)
final_data_promo_grouped.head(1)

In [ ]:
final_data_promo_grouped_EQ = final_data_promo_grouped.merge(map_size, on="retailersku", how="left")
final_data_promo_grouped_EQ[['Size', 'lb/oz', 'Size lb', 'count']] = final_data_promo_grouped_EQ[['Size', 'lb/oz', 'Size lb', 'count']].fillna("Unknown")
final_data_promo_grouped_EQ = final_data_promo_grouped_EQ[['Title', 'title_from_sales', 'retailersku', 'manufacturer', 'brand',
       'manufacturer_general', 'brand_general', 'species_form', 'bag_size',
       'lifestage', 'usage', 'Size', 'lb/oz', 'Size lb', 'count', 'coef',
        'promo_type', 'weekid', "Promo Sales", "Promo Units",  'Promo AvgPrice per Unit' ]]
final_data_promo_grouped_EQ["Promo Volume"] = final_data_promo_grouped_EQ["Promo Units"]*final_data_promo_grouped_EQ["coef"]
final_data_promo_grouped_EQ["Promo AvgPrice per lb"]=final_data_promo_grouped_EQ["Promo Sales"]/final_data_promo_grouped_EQ["Promo Volume"]
final_data_promo_grouped_EQ.fillna(0,inplace=True)

# SNS

In [ ]:
os.chdir(sns_data_path)
sns_data = pd.read_csv("SNS_Scale_Up_2019-06-11.csv")
sns_data["week_range"] = sns_data["week_range"].astype("str")
sns_data['Start'] = sns_data['week_range'].apply(lambda x: x[1:11])
sns_data['End'] = sns_data['week_range'].apply(lambda x: x[14:24])
sns_data['Start'] = sns_data['Start'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))
sns_data['Start_week'] = sns_data['Start']
sns_data['Start_week'] = sns_data['Start_week'].apply(lambda x: datetime.strftime(x, '%U'))
sns_data["year"] = sns_data['Start']
sns_data['year'] = sns_data['year'].apply(lambda x: datetime.strftime(x, '%Y'))
sns_data["Start_week"] = sns_data["Start_week"].astype("int")
sns_data["Start_week"][sns_data["year"]=="2017"]=sns_data["Start_week"]-1
sns_data['weekid']=""
sns_data['weekid'][sns_data["Start_week"]>9]= sns_data['year']+sns_data['Start_week'].astype("str")
sns_data['weekid'][sns_data["Start_week"]<10]= sns_data['year']+"0"+sns_data['Start_week'].astype("str")
sns_data = sns_data[['weekid', 'Asin', 'SNS_Quantity', 'SNS_Net_Sales']]
sns_data.rename(columns={'Asin':"retailersku"},inplace = True)
#final_sns_data["SNS_Quantity"] = final_sns_data["SNS_Quantity"]*-1
#final_sns_data["SNS_Net_Sales"] = final_sns_data["SNS_Net_Sales"]*-1
#final_sns_data.rename(columns={'SNS_Quantity':"volume", 'SNS_Net_Sales':"estimatesales" },inplace = True)
sns_data.rename(columns={'SNS_Quantity':"SNS Units", 'SNS_Net_Sales':"SNS Sales" },inplace = True)
sns_data.drop_duplicates(inplace=True)

In [ ]:
sns_data["SNS Sales"].sum()

# Drop Treats

In [ ]:
sns_data["promo_type"]="None"
sns_data=sns_data.merge(map_title, on="retailersku")
sns_data["retailersku"].replace(to_replace="0789331144", value="789331144", inplace = True)
sns_data_with_attr = sns_data.merge(map_atributes, on="retailersku", how="left")
sns_data_without_treats = sns_data_with_attr[~(sns_data_with_attr["species_form"].str.contains("drop", na=False))]
sns_data_with_treats = sns_data_with_attr[(sns_data_with_attr["species_form"].str.contains("drop", na=False))]
len(sns_data_without_treats)

# Check&CSV

In [ ]:
print(sns_data["SNS Units"].sum()-sns_data_with_attr["SNS Units"].sum())
print(sns_data_without_treats["SNS Units"].sum()+sns_data_with_treats["SNS Units"].sum()-sns_data_with_attr["SNS Units"].sum())

In [ ]:
final_data_sns = sns_data_without_treats[["Title",'title_from_sales',"retailersku",'manufacturer', 'brand','manufacturer_general', 'brand_general', 'species_form',
                                            'bag_size', 'lifestage', 'usage',"promo_type", 'weekid',  "SNS Units", "SNS Sales"]]

In [ ]:
final_data_sns_grouped = final_data_sns.groupby(['Title','title_from_sales',"retailersku",'manufacturer', 'brand','manufacturer_general', 'brand_general', 
                                                     'species_form', 'bag_size', 'lifestage', 'usage',"promo_type", "weekid"]).max()
final_data_sns_grouped["SNS AvgPrice per Unit"]=final_data_sns_grouped["SNS Sales"]/final_data_sns_grouped["SNS Units"]
final_data_sns_grouped.reset_index(inplace =True)
final_data_sns_grouped.head(1)

In [ ]:
final_data_sns_grouped_EQ = final_data_sns_grouped.merge(map_size, on="retailersku", how="left")
final_data_sns_grouped_EQ[['Size', 'lb/oz', 'Size lb', 'count']] = final_data_sns_grouped_EQ[['Size', 'lb/oz', 'Size lb', 'count']].fillna("Unknown")
final_data_sns_grouped_EQ = final_data_sns_grouped_EQ[['Title', 'title_from_sales', 'retailersku', 'manufacturer', 'brand',
       'manufacturer_general', 'brand_general', 'species_form', 'bag_size',
       'lifestage', 'usage',
       'Size', 'lb/oz', 'Size lb', 'count', 'coef', 'promo_type', 'weekid', 
       'SNS Sales', 'SNS Units', 'SNS AvgPrice per Unit' ]]
final_data_sns_grouped_EQ["SNS Volume"] = final_data_sns_grouped_EQ["SNS Units"]*final_data_sns_grouped_EQ["coef"]
final_data_sns_grouped_EQ["SNS AvgPrice per lb"]=final_data_sns_grouped_EQ["SNS Sales"]/final_data_sns_grouped_EQ["SNS Volume"]
final_data_sns_grouped_EQ.fillna(0,inplace=True)

In [ ]:
sns_sales_match.shape

In [ ]:
final_data_sales["Total Sales"].sum()

In [ ]:
final_data_sns_grouped_EQ.head(1)
final_data_sales_grouped_EQ.head(1)
final_data_promo_grouped_EQ.head(1)

In [ ]:
promo_sales_match = final_data_promo_grouped_EQ.merge(final_data_sales_grouped_EQ,on=['Title', 'title_from_sales', 'retailersku', 'manufacturer', 'brand',
       'manufacturer_general', 'brand_general', 'species_form', 'bag_size',
       'lifestage', 'usage', 'Size', 'lb/oz', 'Size lb', 'count', 'coef',
        'weekid'], how="left")
promo_sales_match["Promo_min_Sales"]=promo_sales_match["Total Sales"] - promo_sales_match["Promo Sales"]
promo_sales_match["Promo_min_Sales_U"]=promo_sales_match["Sale Units"] - promo_sales_match["Promo Units"]
promo_sales_match["Promo_min_Sales_P"]=promo_sales_match["AvgPrice per Unit"] - promo_sales_match["Promo AvgPrice per Unit"]
promo_sales_match.to_csv(to_save+"promo_sales_match.csv")

In [ ]:
sns_sales_match = final_data_sns_grouped_EQ.merge(final_data_sales_grouped_EQ,on=['Title', 'title_from_sales', 'retailersku', 'manufacturer', 'brand',
       'manufacturer_general', 'brand_general', 'species_form', 'bag_size',
       'lifestage', 'usage', 'Size', 'lb/oz', 'Size lb', 'count', 'coef',
        'weekid'], how="left")
sns_sales_match["SNS_min_Sales"]=sns_sales_match["Total Sales"] - sns_sales_match["SNS Sales"]
sns_sales_match["SNS_min_Sales_U"]=sns_sales_match["Sale Units"] - sns_sales_match["SNS Units"]
sns_sales_match["SNS_min_Sales_P"]=sns_sales_match["AvgPrice per Unit"] - sns_sales_match["SNS AvgPrice per Unit"]
sns_sales_match.to_csv(to_save+"sns_sales_match.csv")

# Melt all

In [ ]:
final_data_sales_melt = pd.melt(final_data_sales_grouped_EQ, id_vars = ['Title', 'title_from_sales', 'retailersku', 'manufacturer', 'brand','manufacturer_general', 'brand_general',
                                            'species_form', 'bag_size', 'lifestage', 'usage','Size', 'lb/oz', 'Size lb', 'count', 'coef',"promo_type", 'weekid'],
                                          var_name ="Fact", value_name="Values")

In [ ]:
final_data_promo_melt = pd.melt(final_data_promo_grouped_EQ, id_vars = ['Title', 'title_from_sales', 'retailersku', 'manufacturer', 'brand','manufacturer_general', 'brand_general',
                                            'species_form', 'bag_size', 'lifestage', 'usage','Size', 'lb/oz', 'Size lb', 'count', 'coef',"promo_type", 'weekid'],
                                          var_name ="Fact", value_name="Values")

In [ ]:
final_data_sns_melt = pd.melt(final_data_sns_grouped_EQ, id_vars = ['Title', 'title_from_sales', 'retailersku', 'manufacturer', 'brand','manufacturer_general', 'brand_general',
                                            'species_form', 'bag_size', 'lifestage', 'usage','Size', 'lb/oz', 'Size lb', 'count', 'coef',"promo_type", 'weekid'],
                                          var_name ="Fact", value_name="Values")

In [ ]:
all_df = pd.concat([final_data_sales_melt,final_data_promo_melt,final_data_sns_melt])

In [ ]:
all_df.to_csv(to_save+"all_df_v2.csv")